In [11]:
import pandas as pd
import tqdm as tqdm
import pandas as pd
import numpy as np
import os
import librosa

label = pd.read_csv(r'C:\Users\Ramstein\Documents\SmartsuitStudioProjects\Sample project\Export\scene-1\SuddenCrouch_HUMANIK_E46.csv')
df = label.drop(['Timestamp'], axis=1)
dfn = df.to_numpy().astype(np.float32)
print(dfn.shape)

(652, 441)


In [85]:
np.save('label.npy', dfn)

In [188]:
import pandas as pd
import tqdm as tqdm
import pandas as pd
import numpy as np
import os
import librosa

def data_to_spec(data):
    d = librosa.stft(data)   # wav to short-term-fourier-transform
    return librosa.power_to_db(np.abs(d)**2, ref=np.median)

def file_to_spec(filename, i):
    y, sr = librosa.load(filename,sr=16000, offset=(0.03333333333)*i, duration=0.03333333333)
    print(sr)
    return data_to_spec(y)

Path = r'C:\Conv\wavs'
# files = os.listdir(path)
files=['Conv12.wav']
arr = []

for file in tqdm.tqdm(files):
    file = os.path.join(Path, file)
    duration = librosa.core.get_duration(filename=file) 
    print('duration:', duration )
    Slice=int((duration*1000)//33.3333333333)

    for i in range(Slice):
        try:
            d = file_to_spec(file, i)
            d = d.transpose()
        except Exception as e:print(e);continue
        arr.append(d)
    break

data_n = np.array(arr) /255
print(data_n.shape)




























  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

duration: 302.46
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000

KeyboardInterrupt: 

In [185]:
data_n.shape

(9073, 2, 1025)

In [86]:
# np.save('train.npy', data[:652])

In [93]:
import keras
import tensorflow as tf
from keras import backend as k
from tensorflow.keras.layers import Activation, Dense, Dropout, Conv2D, \
                         Flatten, MaxPooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau,ModelCheckpoint,TensorBoard,ProgbarLogger
from sklearn.model_selection import train_test_split
import librosa
import librosa.display
import numpy as np
import pandas as pd
import random
import warnings
warnings.filterwarnings('ignore')

#object serialization
import _pickle as cPickle  #python 3 change
import os  

%matplotlib inline


train = np.load('/kaggle/input/audio-mfcc-test/train.npy')
dfn = np.load('/kaggle/input/audio-mfcc-test/label.npy')

#split up test into test and validation 
X_train, X_val, y_train, y_val = train_test_split(train, dfn, test_size=0.30, random_state=42)

X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.40, random_state=42)

print ("test ",X_train.shape, len(y_train))
print ("valid ", X_val.shape, len(y_val))

test  (456, 33, 1025) 456
valid  (196, 33, 1025) 196


In [ ]:
%%time
%%memit

# NOTE:
# Increase number if epochs from  1 to 60 or 100 for higher prediction accuracy
# default is set to 1 for faster commit 
MAX_EPOCHS=3
MAX_BATCH_SIZE=23            
# learning rate reduction rate 
MAX_PATIENT=2  

# saved model checkpoint file
best_model_file="./best_model_trained.hdf5"

# callbacks
# removed EarlyStopping(patience=MAX_PATIENT)
callback=[ReduceLROnPlateau(patience=MAX_PATIENT, verbose=1),
          ModelCheckpoint(filepath=best_model_file, monitor='loss', verbose=1, save_best_only=True)]

#compile
model.compile(optimizer="Adam",loss="categorical_crossentropy",metrics=['accuracy'])

#train
print('training started.... please wait!')
history = model.fit(x=X_train, y=y_train,
                    epochs=MAX_EPOCHS,
                    batch_size=MAX_BATCH_SIZE, 
                    verbose=1,
                    validation_data= (X_val, y_val), 
                    callbacks=callback)
print('training finished')

# quick evaludate model
print('Evaluate model with test data')
score = model.evaluate(x=X_test,y=y_test)

print('test loss:', score[0])
print('test accuracy:', score[1])

In [ ]:
%%time
%%memit

import matplotlib.pyplot as plt
#Plot loss and accuracy for the training and validation set.
def plot_history(history):
    loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' not in s]
    val_loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' in s]
    acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' not in s]
    val_acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' in s]
    if len(loss_list) == 0:
        print('Loss is missing in history')
        return 
    plt.figure(figsize=(22,10))
    ## As loss always exists
    epochs = range(1,len(history.history[loss_list[0]]) + 1)
    ## Accuracy
    plt.figure(221, figsize=(20,10))
    ## Accuracy
    # plt.figure(2,figsize=(14,5))
    plt.subplot(221, title='Accuracy')
    for l in acc_list:
        plt.plot(epochs, history.history[l], 'b', label='Training accuracy (' + str(format(history.history[l][-1],'.5f'))+')')
    for l in val_acc_list:    
        plt.plot(epochs, history.history[l], 'g', label='Validation accuracy (' + str(format(history.history[l][-1],'.5f'))+')')
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    ## Loss
    plt.subplot(222, title='Loss')
    for l in loss_list:
        plt.plot(epochs, history.history[l], 'b', label='Training loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    for l in val_loss_list:
        plt.plot(epochs, history.history[l], 'g', label='Validation loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))    
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

# plot history
plot_history(history)